In [10]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['enfsi','lfw','forenface'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

In [11]:
from sql_face.sqldb import SQLDataBase

database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [12]:






import pandas as pd
from tqdm import tqdm
from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage

session = database.session



# generamos un datafame con la información que necesitamos para los registros sin quality
# hacemos una query para actualizar los registros que no tienen quality


df_a = pd.read_sql_query(session.query(QualityImage,FaceImage.croppedImage_id)
    .join(FaceImage)
    .filter(QualityImage.quality != None,FaceImage.embeddingModel_id == 1).statement, con=session.bind)


sql_b = (session.query(QualityImage, FaceImage)
    .join(FaceImage)
    .filter(QualityImage.quality == None).limit(200000).all())
n1 = 1000
n = n1
for Q,F in tqdm(sql_b):
    #recorremos el resultado y actualizamos el objeto QualityImage


    qm = Q.qualityModel_id
    em = F.embeddingModel_id
    ci = F.croppedImage_id

    df_r = df_a.loc[(df_a['qualityModel_id'] == qm) & (df_a['croppedImage_id'] == ci) ]
    assert len(df_r) == 1
    Q.quality = float(df_r.quality)
    if not n:
        session.commit()
        n = n1
    else:
        n-=1
session.commit()

0it [00:00, ?it/s]
